# Imports

In [ ]:
# import pandas as pd
# df = pd.read_csv('data\Data\musicFeatuers.csv')

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()

# # df['label'] =  label_encoder.fit_transform(df['label'])

# df = df.drop(['filename'], axis=1)



# X = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var']]

# y = df['label']

# y = df[['label']]
# X = df[df.columns.difference(['label'])]

# from sklearn.model_selection import train_test_split

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1)
# ## train a tree model
# sk_dt.fit(X_train, y_train)

# from sklearn.linear_model import LinearRegression
# # model = LinearRegression()
# # model.fit(X_train, y_train)


# from sklearn.metrics import classification_report

# predictions = sk_dt.predict(X_test)


# print(classification_report(y_test, predictions))

# y_pred = sk_dt.predict(X_test)



# from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(Y_test,y_pred)
# print(mse)


In [ ]:
# %pip install librosa --user

In [ ]:
import xgboost
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import librosa
import numpy as np


import warnings
warnings.filterwarnings('ignore')
# ## create the classifier object leaving all params to default values except random_state
# sk_dt = DecisionTreeClassifier(random_state=1)


# Model

In [ ]:
df = pd.read_csv(r'..\data\Data\features_3_sec.csv')

# df = df.drop(['filename'], axis=1)

# df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','label']]

df = df[['chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


df['label'] =  label_encoder.fit_transform(df['label'])

print(label_encoder.classes_)

y = df[['label']]
X = df[df.columns.difference(['label'])]


## split both X and y using a ratio of 70% training - 30% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(y_train), len(y_test))

target_name = ['blues', 'classical', 'country', 'disco', 'hiphop' ,'jazz' ,'metal', 'pop','reggae' ,'rock']


In [ ]:


xgb = xgboost.XGBClassifier(n_estimators=1000,enable_catergorical=True,learning_rate=0.05)
xgb.fit(X_train, y_train)

## make predictions on the test portion (predict the labels of the rows from the test portion of X)
predictionsXgb = xgb.predict(X_test)



print(classification_report(y_test, predictionsXgb, target_names=target_name))
## can also output the confusion matrix
# cm = confusion_matrix(y_test, predictions)
# print(cm)

print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsXgb))

cols_when_model_builds = xgb.feature_names_in_


In [ ]:
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold


# k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# cv_scores = cross_val_score(xgb, X, y, cv=k_fold, scoring="accuracy")

# for i,score in enumerate(cv_scores, start=1):
#     print(f'Fold {i}: {score:.4f}')

    

In [ ]:
from sklearn.ensemble import RandomForestClassifier


forest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
forest.fit(X_train, y_train)
predictionsForest = forest.predict(X_test)

print(classification_report(y_test, predictionsForest, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsForest))



In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictionsDTC = dtc.predict(X_test)

print(classification_report(y_test, predictionsDTC, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsDTC))

In [ ]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier

scaledX = preprocessing.scale(X_train)
scaledt = preprocessing.scale(X_test)


KNC = KNeighborsClassifier(n_neighbors=1)
KNC.fit(scaledX, y_train)
predictionsKNC = KNC.predict(scaledt)

print(classification_report(y_test, predictionsKNC, target_names=target_name))
print("Accuracy: " ,metrics.accuracy_score(y_test, predictionsKNC))


# Find Similar

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

def find_sim(data):
    placeHoldername = 'test'
    data['filename'] = placeHoldername

    df_sim = pd.read_csv(r'..\data\Data\features_30_sec.csv')

    df_sim = df_sim[['filename','chroma_stft_mean','chroma_stft_var','rms_mean','rms_var','spectral_centroid_mean','spectral_centroid_var','spectral_bandwidth_mean','spectral_bandwidth_var','rolloff_mean','rolloff_var','zero_crossing_rate_mean','zero_crossing_rate_var','harmony_mean','harmony_var','tempo','label']]


    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df_sim['label'] = df_sim['label'].astype("string")
    df_sim['label'] =  label_encoder.fit_transform(df_sim['label'])




    combined_df = pd.concat([df_sim, data], ignore_index=True)

    combined_df = combined_df.set_index('filename')

 
    labels = combined_df[['label']]

    
    scaled = preprocessing.scale(combined_df)
    similarity = cosine_similarity(scaled)
    sim_df_labels = pd.DataFrame(similarity)
    sim_df_names = sim_df_labels.set_index(labels.index)
    sim_df_names.columns = labels.index

    series = sim_df_names[placeHoldername].sort_values(ascending=False)
    series = series.drop(placeHoldername)
    return series.head(3)





# Find Prediction

In [28]:
def find_pred(data, features):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()

    features = features.drop(['filename'], axis=1)
    pred_features =  features.drop(['label'], axis=1)
    pred_features = pd.DataFrame(pred_features)
    for column in pred_features:
        X_train, X_test, y_train, y_test = train_test_split(data, data[column], test_size=0.2)
        model.fit(X_train, y_train)
        prediction = model.predict(features)
        # print(column,":",prediction[0])
        features[column]=prediction[0]
        
    

    # X_train, X_test, y_train, y_test = train_test_split(data, data[predicted_feature], test_size=0.2)


    return features




In [ ]:
def find_pred_poly(data, features, predicted_feature):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression

    features = features.drop(['filename'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(data, data[predicted_feature], test_size=0.2)

    model = LinearRegression()
    model.fit(X_train, y_train)


    prediction = model.predict(features)

    return prediction[0]

# Feature Extractor



In [ ]:
def extract_features(file):
    y, sr = librosa.load(file)

    chroma_sft_mean =  np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    chroma_sft_var =  librosa.feature.chroma_stft(y=y, sr=sr).var()

    rms_mean = librosa.feature.rms(y=y).mean()
    rms_var = librosa.feature.rms(y=y).var()

    spectral_centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_centroid_var = librosa.feature.spectral_centroid(y=y, sr=sr).var()

    spectral_bandwith_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spectral_bandwith_var = librosa.feature.spectral_bandwidth(y=y, sr=sr).var()

    rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    rolloff_var = librosa.feature.spectral_rolloff(y=y, sr=sr).var()

    zero_crossing_rate_mean = librosa.feature.zero_crossing_rate(y=y).mean()
    zero_crossing_rate_var = librosa.feature.zero_crossing_rate(y=y).var()


    harmony_mean = librosa.effects.harmonic(y).mean()
    harmony_var = librosa.effects.harmonic(y).var()

    tempo = librosa.feature.tempo(y=y, sr=sr)[0]

    

    print("chroma_sft_mean: ",chroma_sft_mean)
    print("chroma_sft_var: ",chroma_sft_var)
    print("rms_mean: ",rms_mean)
    print("rms_var: ",rms_var)
    print("spectral_centroid_mean: ",spectral_centroid_mean)
    print("spectral_centroid_var: ",spectral_centroid_var)
    print("spectral_bandwith_mean: ",spectral_bandwith_mean)
    print("spectral_bandwith_var: ",spectral_bandwith_var)
    print("rolloff_mean: ",rolloff_mean)
    print("rolloff_var: ",rolloff_var)
    print("zero_crossing_rate_mean: ",zero_crossing_rate_mean)
    print("zero_crossing_rate_var: ",zero_crossing_rate_var)
    print("harmony_mean: ",harmony_mean)
    print("harmony_var: ",harmony_var)
    print("tempo: ",tempo)

    features = pd.DataFrame({'chroma_stft_mean':[chroma_sft_mean],'chroma_stft_var':[chroma_sft_var],'rms_mean':[rms_mean],'rms_var':[rms_var],'spectral_centroid_mean':[spectral_centroid_mean],
                             'spectral_centroid_var':[spectral_centroid_var],'spectral_bandwidth_mean':[spectral_bandwith_mean],'spectral_bandwidth_var':[spectral_bandwith_var],
                             'rolloff_mean':[rolloff_mean],'rolloff_var':[rolloff_var],'zero_crossing_rate_mean':[zero_crossing_rate_mean],'zero_crossing_rate_var':[zero_crossing_rate_var],
                             'harmony_mean':[harmony_mean],'harmony_var':[harmony_var],'tempo':[tempo],})
    

    features = features.reindex(columns=cols_when_model_builds)

    # features = features.reshape(-1,1)
    return features



In [ ]:
def confidence_score(proba):
    from collections import Counter
    confi = {}
    i=0
    for val in proba[0]:
        rounded = round(val *100,2)
        # print(rounded ,"% :",target_name[i])
        confi[target_name[i]] = rounded
        i = i+1

    k = Counter(confi)
    
    # Finding 3 highest values
    high = k.most_common(3) 
    
    for i in high:
        print(i[0]," :",i[1]," ")

# Main

* 'blues':0, 'classical':1, 'country':2, 'disco':3, 'hiphop':4 ,'jazz':5 ,'metal':6, 'pop':7,'reggae':8 ,'rock':9 

In [ ]:
features1 =  extract_features(r"..\music\Beyoncé-Single Ladies.mp3")
features2 =  extract_features(r"..\music\Outkast-Hey Ya!.mp3")
features3=  extract_features(r"../music/Taylor Swift-Bad Blood.mp3")


print('---------------------------------------------------')

genre1 = xgb.predict(features1)
genreP1 = xgb.predict_proba(features1)
genre2 = xgb.predict(features2)
genreP2 = xgb.predict_proba(features2)

genre3 = xgb.predict(features3)
genreP3 = xgb.predict_proba(features3)

features1['label'] = genre1[0]
features2['label'] = genre2[0]
features3['label'] = genre3[0]

label1 = label_encoder.inverse_transform(genre1)[0]
label2 = label_encoder.inverse_transform(genre2)[0]
label3 = label_encoder.inverse_transform(genre3)[0]

confidence_score(genreP1)
print(label1)

print('---------------------------------------------------')

confidence_score(genreP2)
print(label2)

print('---------------------------------------------------')

confidence_score(genreP3)
print(label3)



In [ ]:
print("Similiar Songs:",find_sim(features1))
print('---------------------------------------------------')

new_features = features1
new_features['tempo'] = 400
print(find_sim(new_features))
print('---------------------------------------------------')




In [30]:
combined_df = pd.concat([features1, features2], ignore_index=True)
combined_df = pd.concat([combined_df, features3], ignore_index=True)
combined_df = combined_df.drop(['filename'], axis=1)

feature = 'tempo'
pred = find_pred(combined_df, features1)
print("Predicted feature: ",pred)
# pred_features = features1
# pred_features[feature] = pred
print("Predicted Song: " ,find_sim(pred))

Predicted feature:     chroma_stft_mean  chroma_stft_var  harmony_mean  harmony_var  rms_mean  \
0          0.444874         0.093534     -0.000002     0.011054  0.163127   

   rms_var  rolloff_mean   rolloff_var  spectral_bandwidth_mean  \
0  0.00346   4432.646114  1.667464e+06              2719.170337   

   spectral_bandwidth_var  spectral_centroid_mean  spectral_centroid_var  \
0            84152.459688              2949.96909          403055.754912   

        tempo  zero_crossing_rate_mean  zero_crossing_rate_var     label  
0  122.819808                 0.083016                0.002968  5.446857  
Predicted Song:  filename
pop.00082.wav       0.643157
hiphop.00029.wav    0.582844
pop.00044.wav       0.563647
Name: test, dtype: float64


In [ ]:
# %pip install tensorflow